# PR0402 - Datos del clima

In [1]:
!hdfs dfs -mkdir /datos_clima

In [ ]:
!hdfs dfs -put city_temperature.csv /datos_clima

In [4]:
!hdfs dfs -ls /datos_clima

Found 1 items
-rw-r--r--   3 root supergroup  140600832 2025-11-18 09:36 /datos_clima/city_temperature.csv


### Ejercicio 1: temperatura máxima por ciudad 

In [2]:
%%writefile mapper.py
#!/usr/bin/env python3

import sys, csv

reader = csv.DictReader(sys.stdin)
for line in reader:
    print(f"{line['City']}\t{line['AvgTemperature']}")

Overwriting mapper.py


In [4]:
%%writefile reducer.py
#!/usr/bin/env python3

import sys, math

current_city = None
max_temp = -math.inf
cities = set()

for line in sys.stdin:
    city, temp = line.strip().split("\t", 1)
    
    if current_city == city:
        cities.add(city)
        if temp > max_temp:
            max_temp = temp
    else:
        print(f"{current_city}\t{','.join(sorted(cities))}")
        max_temp = -math.inf
        current_city = city
        cities = {city}
        
if current_city:
    print(f"{current_city}\t{','.join(sorted(cities))}")

Overwriting reducer.py


In [7]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper.py \
-file reducer.py \
-mapper mapper.py \
-reducer reducer.py \
-input /datos_clima/city_temperature.csv \
-output /salida_temp

2025-11-18 10:13:56,475 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py, /tmp/hadoop-unjar3575307144873537464/] [] /tmp/streamjob6297585062917088757.jar tmpDir=null
2025-11-18 10:13:57,733 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.2:8032
2025-11-18 10:13:57,999 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.2:8032
2025-11-18 10:13:58,330 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763458450721_0005
2025-11-18 10:13:58,935 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-18 10:13:58,963 INFO net.NetworkTopology: Adding a new node: /default-rack/172.19.0.3:9866
2025-11-18 10:13:58,964 INFO net.NetworkTopology: Adding a new node: /default-rack/172.19.0.4:9866
2025-11-18 10:13:58,964 INFO net.NetworkTo

In [6]:
!hdfs dfs -rmdir /salida_temp

In [ ]:
!cat city_temperature.csv | python3 mapper.py | sort | python3 reducer.py